# 基于二项分布的双爆配比分析——1：2真的是最佳配比吗？降低暴击率可以降低凹度？


## 1. 从期望的角度出发，但是继续深入
假设:可分配的总双爆词条数相同,即总和一致.而在原神的圣遗物、武器词条设计下,词条数一定,可以分配的暴击和暴伤比为1:2.由基本不等式可以得到,在连续情况下,伤害期望的最大值正是在暴击和暴伤的比为1:2时取得.这本身没有任何问题,但是,这里求得的是伤害期望.对于一次具体的深渊实战,由于攻击的次数并不是无穷大,甚至不会是一个特别大的值，一次战斗伤害的总值实际上服从一个特定的随机分布.对于一次攻击是否暴击,可以看做是一次伯努利试验,而一场战斗实际上是多次重复的伯努利试验.因此,暴击次数可以看做是符合二项分布.再结合暴击伤害值,通过程序模拟(懒得算)可以得到具体暴击次数(总伤害)的分布情况.

## 2. 研究分布情况,怎么帮助满星深渊
为了便于进行研究,首先抽象出一个问题的模型.这里只计算伤害的数值对于战斗的影响,排除其他因素.并且将伤害的计算简单化,且只考虑双爆因素.

这里先给出一个具体的例子.加入一个面板可以达到100/200的角色(比如220分+夜兰)，那么他的伤害可以计算为100 + 1 * 200 = 300.并且由于暴击率达到了1,他的伤害不会有任何的浮动.假设用于,满星深渊的时间可以打出100次攻击,那么造成的伤害就是30000,并且这个值不会变化.
但是,如果满星深渊所需要的伤害是30500,甚至31000,这样的角色将永远不可能达到这个要求.

这里这个角色完美遵循了双爆1:2的计算结果配面板,但是却不是最佳的情况.这里如果想要打满深渊,只能采用凹分常见操作,也就是降暴击,比如90/220,来尝试达到30500的伤害量.那么,达成这个值有多困难,压更低的暴击会不会更好.在这里先举几个简单地例子引入,在下一部分进行更多的计算分析.

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from scipy import stats
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [42]:
##达成30500伤害量，220爆伤需要94次暴击
stats.binom.cdf(k = 100,n = 100, p =0.9) - stats.binom.cdf(k = 93,n = 100, p =0.9)
##90暴击的角色在100次攻击中暴击94次及以上的概率

0.11715561543588437

In [43]:
##达成30500伤害量，240爆伤需要86次暴击
stats.binom.cdf(k = 100,n = 100, p =0.8) - stats.binom.cdf(k = 85,n = 100, p =0.8)
##80暴击的角色在100次攻击中暴击86次及以上的概率

0.08044372113805087

In [44]:
##达成31000伤害量，220爆伤需要96次暴击
stats.binom.cdf(k = 100,n = 100, p =0.9) - stats.binom.cdf(k = 95,n = 100, p =0.9)
##90暴击的角色在100次攻击中暴击96次及以上的概率

0.023711082663476768

In [45]:
##达成31000伤害量，240爆伤需要88次暴击
stats.binom.cdf(k = 100,n = 100, p =0.8) - stats.binom.cdf(k = 87,n = 100, p =0.8)
##80暴击的角色在100次攻击中暴击88次及以上的概率

0.02532875322139705

##

##

##

##

##

##

##

##

##

##

##

##

##

##

##

##

##

##

##

##

## 3. 从具体的值出发，分析双爆配比降低凹度的最佳方案
本部分有两个目标：

1.通过对代入具体数据的高双爆和低双爆的情况(由于暴击存在上限，猜测可能会出现差异).分析一下达到某个特定伤害值.在各自赛道不同暴击值的概率.在什么特定需求下,更需要降低暴击率?怎样安排双爆,可以用更高的概率打出更高的伤害？

2.对于一个特定的伤害阈值和提升暴伤占比的面板,需要多少凹度才能达到要求.给定一定的凹深渊次数,有多少的把握可以凹满深渊?

In [85]:
############################################################################
#times = 攻击次数
#CritRate 1/2 = 第一/二组暴击率 （e.g:0.5/0.6)
#CritDamage 1/2 = 第一/二组暴击伤害(e.g:150/180)
#range1/2_h = 第一/二组起始暴击占比
#range1/2_t = 第一/二组终止暴击占比
#输出格式：实际暴击率 达到此暴击率及以上的几率 达到此暴击率以上时的期望伤害
############################################################################

times = 100
CritRate1 = 0.95
CritRate2 = 0.85
CritDamage1 = 210
CritDamage2 = 230
range1_h = 0.8
range1_t = 1
range2_h = 0.8
range2_t =1
CritDamage1 = CritDamage1 / 100
CritDamage2 = CritDamage2 / 100

x = np.arange(0,times + 1)
crit01 = stats.binom(times,CritRate1)
crit1 = crit01.pmf(x)
crit02 = stats.binom(times,CritRate2)
crit2 = crit02.pmf(x)

damage01 = np.sum(crit1 * x * CritDamage1)/times * 100 + 100  
damage02 = np.sum(crit2 * x * CritDamage2)/times * 100 + 100  
print(round(damage01,0))
print(round(damage02,0))

damageG1 = []
damageG2 = []
chanceG1 = []
chanceG2 = []
for i in range(int(range1_h * times) ,int(range1_t * times) + 1):
    chance1 = stats.binom.cdf(k = times,n = times, p =CritRate1) - stats.binom.cdf(k = i -1,n = times, p =CritRate1)
    damage01high = ( np.sum(crit1[i:times+1]/chance1 * x[i:times+1] * CritDamage1) /times * 100 + 100 )
    damage1 = 100 + i /times * 100 * CritDamage1
    print(round(i/times * 100,1),round(chance1,6),round(damage1,3),round(damage01high,3))
    damageG1.append(damage1)
    chanceG1.append(chance1)
print('\n')

for i in range(int(range2_h * times) ,int(range2_t * times) + 1):
    chance2 = stats.binom.cdf(k = times,n = times, p =CritRate2 ) - stats.binom.cdf(k = i -1,n = times, p =CritRate2 )
    damage02high = ( np.sum(crit2[i:times+1]/chance2 * x[i:times+1] * CritDamage2) /times * 100 + 100 )
    damage2 = 100 + i /times * 100 * CritDamage2
    print(round(i/times * 100,1),round(chance2,6),round(damage2,3),round(damage02high,3))
    damageG2.append(damage2)
    chanceG2.append(chance2)
    
line1 = go.Scatter(x = damageG1, y = chanceG1, name = 'Data1')
line2 = go.Scatter(x = damageG2, y = chanceG2, name = 'Data2')
fig = go.Figure([line1, line2])
fig.update_layout(
    title = "Probability vs Damage plot",
    xaxis_title = "Damage",
    yaxis_title = "Probability"
)
fig.show()

299.0
296.0
80.0 1.0 268.0 299.5
81.0 1.0 270.1 299.5
82.0 0.999999 272.2 299.5
83.0 0.999998 274.3 299.5
84.0 0.999991 276.4 299.5
85.0 0.999963 278.5 299.501
86.0 0.999864 280.6 299.503
87.0 0.999537 282.7 299.509
88.0 0.998536 284.8 299.526
89.0 0.995726 286.9 299.568
90.0 0.988528 289.0 299.66
91.0 0.971812 291.1 299.843
92.0 0.93691 293.2 300.169
93.0 0.87204 295.3 300.687
94.0 0.766014 297.4 301.433
95.0 0.615999 299.5 302.415
96.0 0.435981 301.6 303.619
97.0 0.257839 303.7 305.013
98.0 0.118263 305.8 306.564
99.0 0.037081 307.9 308.235
100.0 0.005921 310.0 310.0


80.0 0.93368 284.0 296.689
81.0 0.893456 286.3 297.26
82.0 0.837175 288.6 297.997
83.0 0.763277 290.9 298.907
84.0 0.672463 293.2 299.988
85.0 0.568315 295.5 301.232
86.0 0.457224 297.8 302.625
87.0 0.347425 300.1 304.15
88.0 0.247302 302.4 305.79
89.0 0.163486 304.7 307.527
90.0 0.099447 307.0 309.348
91.0 0.055095 309.3 311.238
92.0 0.027476 311.6 313.187
93.0 0.012165 313.9 315.184
94.0 0.004702 316.2 317.221
95.0 0

In [67]:
stats.binom_test(4,50,0.025329) ##对次数的假设检验

0.03769972074026291

In [77]:
stats.binom.ppf(0.01,166,0.027476)  ##暴击次数在一定范围内的把握 ##也可用作出分次数在一定范围内的把握

1.0

In [80]:
stats.binom.ppf(0.01,192,0.023711) ##n的取值应为lg(\alpha)/lg(1-p)

1.0

##

##

##

##

##

##

##

##

##

##

##

##

##

##

##

##

##

##

##

##






































## 小结
1.在需要达到更高伤害的时候(超过一定范围),并且双暴较高时，攻击次数不多时，降低暴击 增加暴伤（不变其他条件） 可以以更高的概率达到目标伤害.

2.深渊有些时候要凹是有意义的，但是还是请提升练度（

In [130]:
?stats.binom

In [10]:
?px.line